In [ ]:
!gdown --id 1-XPi_e6Yd29wpx9mOCrTvesPTJ9ubgZZ #requirements.txt
!gdown --id 13rDhmuFjz3d1NWYm42n7I6pQsyieaazT #sent_score_pair.py
!gdown --id 10GFDrB_C21PA1H7Hjq1SGmyjcD0QehuC #models.py
!gdown --id 10BQYSlGu1CLjFUzFMtwuxgas_Qidjl6L #pairing.py
!gdown --id 13_wkK7rsVgRlYmeoRYdxIT3-E_OJ0IOQ #utils.py
!gdown --id 10ZDlZYoF6XCV8LIscnUVnfg2ZM9tTr-m #eval_sent.py
!gdown --id 1Vss9koo31QGnBr3BYbybe7hC9Ldk06s8 #sent pair txt file
!gdown --id 19uy4-kXXDNYkvP2-Nj44imIosLKFNseE #sentence piece model
!gdown --id 19sHUsvCM_gnHEC27q7ByC-RnV2V7UfL1 # BERT model

Downloading...
From: https://drive.google.com/uc?id=1-XPi_e6Yd29wpx9mOCrTvesPTJ9ubgZZ
To: /content/requirements.txt
100% 68.0/68.0 [00:00<00:00, 54.5kB/s]
Downloading...
From: https://drive.google.com/uc?id=13rDhmuFjz3d1NWYm42n7I6pQsyieaazT
To: /content/score_sentence_pairs.py
100% 2.81k/2.81k [00:00<00:00, 3.75MB/s]
Downloading...
From: https://drive.google.com/uc?id=10GFDrB_C21PA1H7Hjq1SGmyjcD0QehuC
To: /content/models.py
100% 11.3k/11.3k [00:00<00:00, 18.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=10BQYSlGu1CLjFUzFMtwuxgas_Qidjl6L
To: /content/pairing.py
100% 6.09k/6.09k [00:00<00:00, 10.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=13_wkK7rsVgRlYmeoRYdxIT3-E_OJ0IOQ
To: /content/utils.py
100% 5.65k/5.65k [00:00<00:00, 5.26MB/s]
Downloading...
From: https://drive.google.com/uc?id=10ZDlZYoF6XCV8LIscnUVnfg2ZM9tTr-m
To: /content/evaluate_sts.py
100% 11.4k/11.4k [00:00<00:00, 9.84MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Vss9koo31QGnBr3BYbybe7

In [ ]:
!pip install -r /content/requirements.txt

In [ ]:
# !python -u score_sentence_pairs.py --sentence-pair-file /content/sent_pairs.txt --load-file /content/model.para.lc.100.pt  --sp-model /content/paranmt.model --gpu=1

Averaging(
  (sim_loss): MarginRankingLoss()
  (cosine): CosineSimilarity()
  (embedding): Embedding(82983, 1024)
)
opical Imiquimod 5% Cream in the Treatment of Nodular Basal Cell Carcinoma After Initial Treatment With Curettage---after initiating treatment with the cuettage treatment, topical imiquimod 5% cream is used for the treatment of nodular basal cell carcinoma.	0.9265482425689697
opical Imiquimod 5% Cream in the Treatment of Nodular Basal Cell Carcinoma After Initial Treatment With Curettage---the topical imiquimod in the treatment of nodal basal cell following an initial treatment	0.7626658082008362
opical Imiquimod 5% Cream in the Treatment of Nodular Basal Cell Carcinoma After Initial Treatment With Curettage---after initiating treatment with the cuettage treatment is used for the treatment of nodular basal cell carcinoma.	0.7700982689857483
opical Imiquimod 5% Cream in the Treatment of Nodular Basal Cell Carcinoma After Initial Treatment With Curettage---after initiating 

In [ ]:
import argparse
from models import load_model

parser = argparse.ArgumentParser()
parser.add_argument("--load-file", default = "/content/model.para.lc.100.pt",help="path to saved model")
parser.add_argument("--sp-model", default = "/content/paranmt.model",help="sentencepiece model to use")
parser.add_argument("--gpu", default=1, type=int, help="whether to train on gpu")

args = parser.parse_args(args=[])

model, _ = load_model(None, args)
model.eval()

Averaging(
  (sim_loss): MarginRankingLoss()
  (cosine): CosineSimilarity()
  (embedding): Embedding(82983, 1024)
)


Averaging(
  (sim_loss): MarginRankingLoss()
  (cosine): CosineSimilarity()
  (embedding): Embedding(82983, 1024)
)

In [ ]:
import argparse
import numpy as np
from sacremoses import MosesTokenizer
from models import load_model
from sacremoses import MosesTokenizer
from utils import Example

def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

class FileSim:

    def __init__(self):
        self.similarity = lambda s1, s2: np.nan_to_num(cosine(np.nan_to_num(s1), np.nan_to_num(s2)))

    def score(self, params, batcher, s1, s2):
        input1 = [s1]
        input2 = [s2]
        sys_scores = []
        for ii in range(0, len(input1), params.batch_size):
            batch1 = input1[ii:ii + params.batch_size]
            batch2 = input2[ii:ii + params.batch_size]

            # we assume get_batch already throws out the faulty ones
            if len(batch1) == len(batch2) and len(batch1) > 0:
                enc1 = batcher(params, batch1)
                enc2 = batcher(params, batch2)

                for kk in range(enc2.shape[0]):
                    sys_score = self.similarity(enc1[kk], enc2[kk])
                    sys_scores.append(sys_score)

        return sys_scores

def batcher(params, batch):
    new_batch = []
    for p in batch:
        if params.tokenize:
            tok = params.entok.tokenize(p, escape=False)
            p = " ".join(tok)
        if params.lower_case:
            p = p.lower()
        p = params.sp.EncodeAsPieces(p)
        p = " ".join(p)
        p = Example(p, params.lower_case)
        p.populate_embeddings(params.model.vocab, params.model.zero_unk, params.model.ngrams)
        new_batch.append(p)
    x, l = params.model.torchify_batch(new_batch)
    vecs = params.model.encode(x, l)
    return vecs.detach().cpu().numpy()

In [ ]:
def evaluate(args, model, s1, s2):

    entok = MosesTokenizer(lang='en')

    from argparse import Namespace

    new_args = Namespace(batch_size=32, entok=entok, sp=model.sp,
                     params=args, model=model, lower_case=model.args.lower_case,
                     tokenize=model.args.tokenize)
    
    s = FileSim()
    scores = s.score(new_args, batcher, s1, s2)
    return scores

In [ ]:
print(evaluate(args, model, "My cat.", "Stray cat."))

[0.64013505]
